In [47]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k.csv')
df = df[:10]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv',index=False)


In [48]:
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_data_5k.csv')
df = df[:125]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_demo.csv',index=False)

In [1]:
import yaml
from easydict import EasyDict

with open('/hpc2hdd/home/yli106/smiles2mol/config/qm9_default.yml', 'r') as file:
    config = yaml.safe_load(file)

config = EasyDict(config)

In [59]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import argparse
from easydict import EasyDict
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/llama2_7b_chat", trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token # Use the EOS token to pad shorter sequences
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

max_len = 0
for i in tqdm(range(len(df))):
    input_text = df['text'][i]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    if len(input_ids[0]) > max_len:
        max_len = len(input_ids[0])
max_len   

100%|██████████| 200000/200000 [10:49<00:00, 307.90it/s]


1881

In [1]:
import pickle

with open('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/test_data_200.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [26]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_data_5k.csv')